### MAST30034: Applied Data Science Project 2
---
# Naive Ranking: Unweighted Average Rank

Calculate the unweights average rank over all merchants (without segmentation).

In [7]:
# useful imports
import pandas as pd
import numpy as np
import os
import re

os.chdir("../scripts")
import utilities.rank_utilities as RANK

In [8]:
# read the curated data
merchant_df = pd.read_parquet('../data/curated/final_merchant_statistics')

In [9]:
merchant_df.head(5)

,merchant_abn,name,tags,tag,revenue_level,take_rate,tot_revenue,no_orders,avg_daily_rev,avg_value_per_order,avg_daily_order,avg_daily_commission,avg_commission_per_order,sa2_region_count,median_customer_income,returning_customer,mean_spending,std_spending,vip_customer
0,10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",e,0.18,7.032777e+05,3261.0,1162.442498,215.663205,5.390083,2.092396,0.388194,1717.0,800.259547,11.0,231.951752,155.424439,137.0
1,10346855916,Odio Institute,"((equipment, tool, furniture, and appliance r...","equipment, tool, furniture, and appliance ren...",b,3.57,6.775178e+03,6.0,11.198642,1129.196413,0.009917,0.399792,40.312312,7.0,795.400000,0.0,1129.196413,717.988423,0.0
2,10385163239,Sed Et Company,"([florists supplies, nursery stock, and flower...","florists supplies, nursery stock, and flowers",a,6.61,2.519346e+04,74.0,41.642088,340.452204,0.122314,2.752542,22.503891,132.0,800.853287,0.0,345.115933,203.496326,3.0
3,10648956813,Proin Nisl Institute,"[(computeRs, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",a,6.66,1.419600e+06,21981.0,2346.446772,64.583063,36.332231,156.273355,4.301232,2208.0,835.366766,1574.0,98.720466,57.083817,710.0
4,10714068705,Sollicitudin Commodo Ipsum Industries,"([furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",c,2.51,4.355100e+05,3438.0,719.851158,126.675378,5.682645,18.068264,3.179552,1715.0,802.506127,10.0,135.209547,113.737044,145.0


In [10]:
# define the columns to add ranks for
cols_to_rank = set(merchant_df).difference(
    ['merchant_abn', 'name', 'tags', 'tag', 'revenue_level']
)

# iterate through and rank these columns
for colname in cols_to_rank:
    merchant_df = RANK.add_column_rank(merchant_df, colname)

In [11]:
# calculate the uwar for the cols to rank
merchant_df = RANK.average_rank(merchant_df, cols_to_rank)

In [12]:
merchant_df[['name', 'average_rank', 'avg_commission_per_order']].sort_values('average_rank')

,name,average_rank,avg_commission_per_order
635,Sem Magna Company,1.0,131.514364
1321,Phasellus Nulla LLC,2.0,124.687957
2422,Tempus Risus LLC,3.0,120.654822
1157,Mi Corporation,4.0,113.991000
2421,Etiam Limited,5.0,109.109921
...,...,...,...
3762,Curae Foundation,NaN,NaN
3791,Fringilla Porttitor Vulputate PC,NaN,NaN
3878,Elit LLP,NaN,NaN
3904,Cras Convallis Associates,NaN,NaN
